In [26]:
import os
import pandas as pd
import pyodbc
from langchain.chat_models import AzureChatOpenAI
from langchain.agents import create_sql_agent
from langchain.agents.agent_toolkits import SQLDatabaseToolkit
from langchain.sql_database import SQLDatabase
from langchain import SQLDatabaseChain
from langchain.chains import SQLDatabaseSequentialChain
from langchain.agents import AgentExecutor
from langchain.callbacks.manager import CallbackManager

from IPython.display import Markdown, HTML, display  


def printmd(string):
    display(Markdown(string))

In [9]:
import os
os.environ["OPENAI_API_TYPE"] = "azure"
os.environ["OPENAI_API_VERSION"] = "2023-03-15-preview"
os.environ["OPENAI_API_BASE"] = "https://aoaitest3.openai.azure.com/"
os.environ["OPENAI_API_KEY"] = "60bf5d22eb3d4b479e241786edbf2408"


In [5]:
import os  
  
os.environ["SQL_SERVER_USERNAME"] = "vaalt"  
os.environ["SQL_SERVER_ENDPOINT"] = "vaalt.database.windows.net"  
os.environ["SQL_SERVER_PASSWORD"] = "Lardosa!12"  
os.environ["SQL_SERVER_DATABASE"] = "vaalt1"  


In [69]:
from sqlalchemy import create_engine
from sqlalchemy.engine.url import URL

db_config = {  
    'drivername': 'mssql+pyodbc',  
    'username': os.environ["SQL_SERVER_USERNAME"] + '@' + os.environ["SQL_SERVER_ENDPOINT"],  
    'password': os.environ["SQL_SERVER_PASSWORD"],  
    'host': os.environ["SQL_SERVER_ENDPOINT"],  
    'port': 1433,  
    'database': os.environ["SQL_SERVER_DATABASE"],  
    'query': {'driver': 'ODBC Driver 18 for SQL Server'}  
}  


# Create a URL object for connecting to the database
db_url = URL.create(**db_config)

# Print the resulting URL string
# print(db_url)

# Connect to the Azure SQL Database using the URL string
engine = create_engine(db_url)

# Test the connection
try:
    conn = engine.connect()
    print("Connection successful!")
    result = engine.execute("SELECT @@Version")
    for row in result:
        print(row)
    conn.close()
    
except OperationalError:
    print("Connection failed.")

Connection successful!
('Microsoft SQL Azure (RTM) - 12.0.2000.8 \n\tJul  8 2023 12:00:47 \n\tCopyright (C) 2022 Microsoft Corporation\n',)


In [88]:
import pyodbc  
  
server = 'vaalt.database.windows.net'  
database = 'vaalt1'  
username = 'vaalt'  
password = 'Lardosa!12'  
driver = '{ODBC Driver 18 for SQL Server}'  
  
cnxn = pyodbc.connect(f'DRIVER={driver};SERVER={server};PORT=1433;DATABASE={database};UID={username};PWD={password}')  
cursor = cnxn.cursor()  
  
# List all tables  
cursor.execute("SELECT TABLE_SCHEMA, TABLE_NAME FROM INFORMATION_SCHEMA.TABLES;")  
rows = cursor.fetchall()  
if rows:  
    print("Tables:")  
    for row in rows:  
        print(f'{row[0]}.{row[1]}')  
  
cursor.close()  
cnxn.close()  


Tables:
sys.database_firewall_rules
dbo.covidtracking
dbo.Titanic
SalesLT.Customer
SalesLT.ProductModel
SalesLT.vProductModelCatalogDescription
SalesLT.ProductDescription
SalesLT.Product
SalesLT.ProductModelProductDescription
SalesLT.vProductAndDescription
SalesLT.ProductCategory
SalesLT.vGetAllCategories
dbo.BuildVersion
dbo.ErrorLog
SalesLT.Address
SalesLT.CustomerAddress
SalesLT.SalesOrderDetail
SalesLT.SalesOrderHeader


In [84]:
titanic = pd.read_csv("data/titanic.csv").fillna(value=0)

for col in titanic.columns:
    if titanic[col].dtype == "float64":
        titanic.loc[titanic[col] == float("inf"), col] = 1
        titanic.loc[titanic[col] == float("-inf"), col] = -1


In [87]:
# Create dataframe from csv file

cnxn = pyodbc.connect(f'DRIVER={driver};SERVER={server};PORT=1433;DATABASE={database};UID={username};PWD={password}')  
cursor = cnxn.cursor()  

# Insert Dataframe into SQL Server:

# Define the create table statement
create_table_query = """
CREATE TABLE Titanic (
    PassengerId int PRIMARY KEY,
    Survived bit NOT NULL,
    Pclass int NOT NULL,
    Name varchar(100) NOT NULL,
    Sex varchar(10) NOT NULL,
    Age float NULL,
    SibSp int NOT NULL,
    Parch int NOT NULL,
    Ticket varchar(20) NOT NULL,
    Fare float NOT NULL,
    Cabin varchar(20) NULL,
    Embarked char(1) NULL
);
"""

# Execute the statement using cursor.execute
cursor.execute(create_table_query)


for index, row in titanic.iterrows():
     cursor.execute(
    "INSERT INTO dbo.[Titanic] (PassengerId, Survived, Pclass, Name, Sex, Age, SibSp, Parch, Ticket, Fare, Cabin, Embarked) values(?,?,?,?,?,?,?,?,?,?,?,?)",
    row.PassengerId,
    row.Survived,
    row.Pclass,
    row.Name,
    row.Sex,
    row.Age,
    row.SibSp,
    row.Parch,
    row.Ticket,
    row.Fare,
    row.Cabin,
    row.Embarked
)
cnxn.commit()
cursor.close()


In [113]:
cnxn = pyodbc.connect(f'DRIVER={driver};SERVER={server};PORT=1433;DATABASE={database};UID={username};PWD={password}')  
cursor = cnxn.cursor()  
print(cursor.execute("select * from dbo.[Titanic]"))
results = cursor.fetchall()

for row in results:
    print(row)

(1, False, 3, 'Braund, Mr. Owen Harris', 'male', 22.0, 1, 0, 'A/5 21171', 7.25, '0', 'S')
(2, True, 1, 'Cumings, Mrs. John Bradley (Florence Briggs Thayer)', 'female', 38.0, 1, 0, 'PC 17599', 71.2833, 'C85', 'C')
(3, True, 3, 'Heikkinen, Miss. Laina', 'female', 26.0, 0, 0, 'STON/O2. 3101282', 7.925, '0', 'S')
(4, True, 1, 'Futrelle, Mrs. Jacques Heath (Lily May Peel)', 'female', 35.0, 1, 0, '113803', 53.1, 'C123', 'S')
(5, False, 3, 'Allen, Mr. William Henry', 'male', 35.0, 0, 0, '373450', 8.05, '0', 'S')
(6, False, 3, 'Moran, Mr. James', 'male', 0.0, 0, 0, '330877', 8.4583, '0', 'Q')
(7, False, 1, 'McCarthy, Mr. Timothy J', 'male', 54.0, 0, 0, '17463', 51.8625, 'E46', 'S')
(8, False, 3, 'Palsson, Master. Gosta Leonard', 'male', 2.0, 3, 1, '349909', 21.075, '0', 'S')
(9, True, 3, 'Johnson, Mrs. Oscar W (Elisabeth Vilhelmina Berg)', 'female', 27.0, 0, 2, '347742', 11.1333, '0', 'S')
(10, True, 2, 'Nasser, Mrs. Nicholas (Adele Achem)', 'female', 14.0, 1, 0, '237736', 30.0708, '0', 'C')
(

In [106]:
# Create or LLM Langchain object using GPT-4 deployment
# Again we need GPT-4. It is necesary in the use of Agents. GPT-35-Turbo will make many mistakes.
llm = AzureChatOpenAI(deployment_name="gpt-4", temperature=0, max_tokens=4000)
#print(llm('tell me a joke'))

In [114]:
db = SQLDatabase.from_uri(db_url)
toolkit = SQLDatabaseToolkit(db=db, llm=llm)

In [93]:
print(db.table_info)


CREATE TABLE [BuildVersion] (
	[SystemInformationID] TINYINT NOT NULL IDENTITY(1,1), 
	[Database Version] NVARCHAR(25) COLLATE SQL_Latin1_General_CP1_CI_AS NOT NULL, 
	[VersionDate] DATETIME NOT NULL, 
	[ModifiedDate] DATETIME NOT NULL DEFAULT (getdate()), 
	CONSTRAINT [PK__BuildVer__35E58ECA9D1F2EAA] PRIMARY KEY ([SystemInformationID])
)

/*
3 rows from BuildVersion table:
SystemInformationID	Database Version	VersionDate	ModifiedDate
1	10.50.91013.00	2009-10-13 00:00:00	2009-10-13 00:00:00
*/


CREATE TABLE [ErrorLog] (
	[ErrorLogID] INTEGER NOT NULL IDENTITY(1,1), 
	[ErrorTime] DATETIME NOT NULL DEFAULT (getdate()), 
	[UserName] NVARCHAR(128) COLLATE SQL_Latin1_General_CP1_CI_AS NOT NULL, 
	[ErrorNumber] INTEGER NOT NULL, 
	[ErrorSeverity] INTEGER NULL, 
	[ErrorState] INTEGER NULL, 
	[ErrorProcedure] NVARCHAR(126) COLLATE SQL_Latin1_General_CP1_CI_AS NULL, 
	[ErrorLine] INTEGER NULL, 
	[ErrorMessage] NVARCHAR(4000) COLLATE SQL_Latin1_General_CP1_CI_AS NOT NULL, 
	CONSTRAINT [PK_Erro

In [115]:
from langchain.agents import AgentExecutor
from langchain.agents.agent_types import AgentType

agent_executor = create_sql_agent(
    llm=llm,
    toolkit=toolkit,
    verbose=True,
    agent_type=AgentType.ZERO_SHOT_REACT_DESCRIPTION,
)

In [110]:
agent_executor.run("how many rows are there in the titanic table?")



> Entering new AgentExecutor chain...
Action: list_tables_sql_db
Action Input: 
Observation: covidtracking, BuildVersion, ErrorLog, Titanic
Thought:I see the Titanic table in the list of tables. I will now write a query to count the number of rows in the Titanic table.
Action: query_checker_sql_db
Action Input: SELECT COUNT(*) FROM Titanic
Observation: SELECT COUNT(*) FROM Titanic
Thought:The query is correct. I will now execute the query to get the number of rows in the Titanic table.
Action: query_sql_db
Action Input: SELECT COUNT(*) FROM Titanic
Observation: [(891,)]
Thought:I now know the final answer
Final Answer: There are 891 rows in the Titanic table.

> Finished chain.


'There are 891 rows in the Titanic table.'

In [111]:
agent_executor.run("what is the name of the oldest survivor of titanic?")



> Entering new AgentExecutor chain...
Action: list_tables_sql_db
Action Input: 
Observation: covidtracking, BuildVersion, ErrorLog, Titanic
Thought:I should check the schema of the Titanic table to see what columns are available.
Action: schema_sql_db
Action Input: Titanic
Observation: 
CREATE TABLE [Titanic] (
	[PassengerId] INTEGER NOT NULL, 
	[Survived] BIT NOT NULL, 
	[Pclass] INTEGER NOT NULL, 
	[Name] VARCHAR(100) COLLATE SQL_Latin1_General_CP1_CI_AS NOT NULL, 
	[Sex] VARCHAR(10) COLLATE SQL_Latin1_General_CP1_CI_AS NOT NULL, 
	[Age] FLOAT(53) NULL, 
	[SibSp] INTEGER NOT NULL, 
	[Parch] INTEGER NOT NULL, 
	[Ticket] VARCHAR(20) COLLATE SQL_Latin1_General_CP1_CI_AS NOT NULL, 
	[Fare] FLOAT(53) NOT NULL, 
	[Cabin] VARCHAR(20) COLLATE SQL_Latin1_General_CP1_CI_AS NULL, 
	[Embarked] CHAR(1) COLLATE SQL_Latin1_General_CP1_CI_AS NULL, 
	CONSTRAINT [PK__Titanic__88915FB0203E0CED] PRIMARY KEY ([PassengerId])
)

/*
3 rows from Titanic table:
PassengerId	Survived	Pclass	Name	Sex	Age	SibSp

Retrying langchain.chat_models.openai.ChatOpenAI.completion_with_retry.<locals>._completion_with_retry in 1.0 seconds as it raised RateLimitError: Requests to the Creates a completion for the chat message Operation under Azure OpenAI API version 2023-03-15-preview have exceeded token rate limit of your current OpenAI S0 pricing tier. Please retry after 12 seconds. Please go here: https://aka.ms/oai/quotaincrease if you would like to further increase the default rate limit..



Observation: [('Barkworth, Mr. Algernon Henry Wilson', 80.0)]
Thought:

Retrying langchain.chat_models.openai.ChatOpenAI.completion_with_retry.<locals>._completion_with_retry in 2.0 seconds as it raised RateLimitError: Requests to the Creates a completion for the chat message Operation under Azure OpenAI API version 2023-03-15-preview have exceeded token rate limit of your current OpenAI S0 pricing tier. Please retry after 11 seconds. Please go here: https://aka.ms/oai/quotaincrease if you would like to further increase the default rate limit..
Retrying langchain.chat_models.openai.ChatOpenAI.completion_with_retry.<locals>._completion_with_retry in 4.0 seconds as it raised RateLimitError: Requests to the Creates a completion for the chat message Operation under Azure OpenAI API version 2023-03-15-preview have exceeded token rate limit of your current OpenAI S0 pricing tier. Please retry after 9 seconds. Please go here: https://aka.ms/oai/quotaincrease if you would like to further increase the default rate limit..
Retrying langchain.chat_models.openai.ChatO

I now know the final answer.
Final Answer: The oldest survivor of the Titanic is Barkworth, Mr. Algernon Henry Wilson, who was 80 years old.

> Finished chain.


'The oldest survivor of the Titanic is Barkworth, Mr. Algernon Henry Wilson, who was 80 years old.'

In [119]:
agent_executor.run("which kind of machine learning model should I use to predict the likelihood of survival in titanic?")



> Entering new AgentExecutor chain...
Action: list_tables_sql_db
Action Input: 
Observation: covidtracking, BuildVersion, ErrorLog, Titanic
Thought:I see a Titanic table, I should check its schema to see if it has relevant information.
Action: schema_sql_db
Action Input: Titanic
Observation: 
CREATE TABLE [Titanic] (
	[PassengerId] INTEGER NOT NULL, 
	[Survived] BIT NOT NULL, 
	[Pclass] INTEGER NOT NULL, 
	[Name] VARCHAR(100) COLLATE SQL_Latin1_General_CP1_CI_AS NOT NULL, 
	[Sex] VARCHAR(10) COLLATE SQL_Latin1_General_CP1_CI_AS NOT NULL, 
	[Age] FLOAT(53) NULL, 
	[SibSp] INTEGER NOT NULL, 
	[Parch] INTEGER NOT NULL, 
	[Ticket] VARCHAR(20) COLLATE SQL_Latin1_General_CP1_CI_AS NOT NULL, 
	[Fare] FLOAT(53) NOT NULL, 
	[Cabin] VARCHAR(20) COLLATE SQL_Latin1_General_CP1_CI_AS NULL, 
	[Embarked] CHAR(1) COLLATE SQL_Latin1_General_CP1_CI_AS NULL, 
	CONSTRAINT [PK__Titanic__88915FB0203E0CED] PRIMARY KEY ([PassengerId])
)

/*
3 rows from Titanic table:
PassengerId	Survived	Pclass	Name	Sex	Age

"I can't directly recommend a specific machine learning model, but you can use the Titanic dataset to train a model. The dataset includes features such as Pclass, Sex, Age, SibSp, Parch, Fare, and Embarked, along with the survival status of each passenger. You can try various classification models like logistic regression, decision trees, or random forests to predict the likelihood of survival."